In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Dense, Flatten, Dropout
import numpy as np


from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomTranslation


In [2]:
# Directory Path

BASE_DIR = 'Dataset'
print(f"Base Directory: {os.listdir(BASE_DIR)}")

data_train='Dataset/train'
print(f"Data Directory: {os.listdir(data_train)}")

data_validation='Dataset/valid'
print(f"Data Directory: {os.listdir(data_validation)}")

Base Directory: ['train', 'valid']
Data Directory: ['Belimbing Sayur', 'Bunga Lawang', 'Jahe', 'Jeruk Nipis', 'Kunyit', 'Lengkuas', 'Mengkudu']
Data Directory: ['Belimbing Sayur', 'Bunga Lawang', 'Jahe', 'Jeruk Nipis', 'Kunyit', 'Lengkuas', 'Mengkudu']


In [3]:
BATCH_SIZE = 128
IMAGE_SIZE = (224, 224)
LABEL_MODE = 'categorical'

# Inisiasi Dataset
# Instantiate the training dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_train,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode=LABEL_MODE
)

# Instantiate the validation dataset
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_validation,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode=LABEL_MODE
)


Found 2055 files belonging to 7 classes.
AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Found 487 files belonging to 7 classes.


In [4]:
SHUFFLE_BUFFER_SIZE = 2000
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (
    train_dataset
    .shuffle(SHUFFLE_BUFFER_SIZE)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

valid_dataset = (
    validation_dataset
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [5]:
FILL_MODE='reflect'
# Membuat layer augmentasi data
data_augmentation = tf.keras.Sequential([
    tf.keras.Input(shape=(224, 224, 3)),
    RandomFlip("horizontal"),# Membalik gambar secara horizontal
    RandomRotation(0.2, fill_mode=FILL_MODE), # Rotasi gambar hingga 20%
    RandomZoom(0.2, fill_mode=FILL_MODE),  # Zoom gambar hingga 20%
    RandomContrast(0.2), # Mengubah kontras gambar hingga 20%
    RandomTranslation(0.1, 0.1, fill_mode=FILL_MODE) # Translasi (geser) gambar hingga 10% horizontal dan vertikal
])

In [6]:
# Load MobileNetV2 tanpa top layer (head), menggunakan bobot ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan layer dari base model agar tidak ikut dilatih
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Bekukan sebagian besar layer awal
    layer.trainable = False


In [7]:
# Model lengkap
model = Sequential([
    data_augmentation,  # Augmentasi data
    base_model,  # MobileNetV2
    Dense(128, activation="relu"),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(7, activation="softmax")  # Jumlah kelas = 7
])

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [9]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Callbacks
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_accuracy = logs.get('accuracy')
        val_accuracy = logs.get('val_accuracy')

        if train_accuracy >= 0.95 and val_accuracy >= 0.85:
            self.model.stop_training = True
            print("\nReached 95% train accuracy and 85% validation accuracy, so cancelling training!")



In [10]:


# Training model dengan data augmentasi
history = model.fit(
    train_dataset,
    epochs=100,
    verbose=1,
    validation_data=valid_dataset,
    callbacks=[EarlyStoppingCallback(), lr_scheduler],

)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 208s 6s/step - accuracy: 0.1552 - loss: 3.7500 - val_accuracy: 0.1602 - val_loss: 2.0062 - learning_rate: 1.0000e-04
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.2102 - loss: 2.2117 - val_accuracy: 0.2053 - val_loss: 1.9283 - learning_rate: 1.0000e-04
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.2167 - loss: 1.9885 - val_accuracy: 0.2033 - val_loss: 1.9068 - learning_rate: 1.0000e-04
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - accuracy: 0.2496 - loss: 1.8773 - val_accuracy: 0.2115 - val_loss: 1.8763 - learning_rate: 1.0000e-04
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 87s 5s/step - accuracy: 0.2584 - loss: 1.8192 - val_accuracy: 0.2074 - val_loss: 1.8918 - learning_rate: 1.0000e-04
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 87s 5s/step - accuracy: 0.2902 - loss: 1.7735 - val_accuracy: 0.2238 - val_loss: 1.9346 - learning_rate: 1.0000e-04
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 92s 5s/step - accuracy: 0.3333

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


In [ ]:
model.save('mobilenetv2_model.h5')


In [ ]:


# Load gambar dan preprocess
img_path = 'path/to/image.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Prediksi
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
print(f"Predicted Class: {predicted_class}")
